# Test Avatar Input Generation

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.objects.log.importer.csv import factory as ocel_import_factory_csv
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.utils import get_happy_path_log, create_flower_model, generate_variant_model, sample_traces
import numpy as np
from tqdm import tqdm
import random

In [13]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})
log = sample_traces(ocel, ocpn, 1000)

In [17]:
happy_path__ocel = get_happy_path_log(filename)
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})
log_happy = sample_traces(ocel, happy_path_ocpn, 5)

In [20]:
ots = ["order","item","delivery"]
flower_ocpn = create_flower_model(filename,ots)
log_flower = sample_traces(ocel, flower_ocpn, 5)

In [25]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ots = ["order","item","delivery"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/order_process_variants/order_variant',object_types = ots )
log_variant = sample_traces(ocel, variant_ocpn, 20)

Processing Variant Nets: 100%|██████████| 12/12 [00:00<00:00, 19988.74it/s]

#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


# O2C Log

### Standard Petri Net

In a first step, we generate a sample from the object-centric petri net.

In [29]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})
log = sample_traces(ocel, ocpn, 5)

Generate the traces: 100%|██████████| 5/5 [00:00<00:00, 1663.61it/s]


In [30]:
log

[['Place Order',
  'Fuel Car',
  'Confirm Order',
  'Payment Reminder',
  'Pick Item',
  'Pay Order',
  'Item out of stock',
  'Reorder Item',
  'Pick Item',
  'Load Cargo',
  'Start Route',
  'End Route'],
 ['Place Order',
  'Confirm Order',
  'Payment Reminder',
  'Payment Reminder',
  'Pick Item',
  'Pay Order',
  'Load Cargo',
  'Item out of stock',
  'Fuel Car',
  'Start Route',
  'End Route'],
 ['Fuel Car',
  'Place Order',
  'Confirm Order',
  'Pick Item',
  'Load Cargo',
  'Payment Reminder',
  'Item out of stock',
  'Pay Order',
  'Reorder Item',
  'Start Route',
  'Pick Item',
  'Load Cargo',
  'End Route'],
 ['Fuel Car',
  'Place Order',
  'Confirm Order',
  'Item out of stock',
  'Pay Order',
  'Pick Item',
  'Reorder Item',
  'Load Cargo',
  'Pick Item',
  'Start Route',
  'End Route'],
 ['Place Order',
  'Fuel Car',
  'Confirm Order',
  'Item out of stock',
  'Pay Order',
  'Reorder Item',
  'Pick Item',
  'Load Cargo',
  'Start Route',
  'End Route']]

In [31]:
# Adapt the log such that each trace is only one complete string separated by commas
activity_strings = log.apply(lambda x: ','.join(x)).tolist()

AttributeError: 'list' object has no attribute 'apply'

In [25]:
activity_strings

['Place Order,Confirm Order,Item out of stock,Pick Item,Pay Order,Reorder Item,Place Order,Confirm Order,Pay Order,Pick Item,Pick Item,Place Order,Confirm Order,Item out of stock,Pick Item,Payment Reminder,Reorder Item,Place Order,Confirm Order,Item out of stock,Reorder Item,Pay Order,Pick Item,Place Order,Confirm Order,Item out of stock,Payment Reminder,Reorder Item,Pick Item,Load Cargo,Place Order,Confirm Order,Item out of stock,Pick Item,Pay Order,Reorder Item,Load Cargo,Place Order,Confirm Order,Pay Order,Pick Item,Pick Item,Load Cargo,Place Order,Confirm Order,Load Cargo,Payment Reminder,Pick Item,Pick Item,Place Order,Confirm Order,Item out of stock,Payment Reminder,Pick Item,Reorder Item,Place Order,Confirm Order,Item out of stock,Item out of stock,Reorder Item,Pay Order,Reorder Item,Place Order,Confirm Order,Item out of stock,Payment Reminder,Pick Item,Reorder Item,Load Cargo,Place Order,Confirm Order,Item out of stock,Payment Reminder,Pick Item,Reorder Item,Load Cargo,Place Or